In [1]:
!pip install python-rake

In [2]:
!pip3 install summa

     |████████████████████████████████| 54 kB 3.3 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54410 sha256=c2638aa36cdd138e744752df64a49b93896dfded95767706c5d9e6b1301a6d80
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
Successfully built summa


In [3]:
!pip install rank_bm25


In [4]:
!pip3 install pymorphy2

     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 8.2 MB 36.7 MB/s 


In [5]:
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [29]:
import pprint

In [6]:
from summa import keywords

In [7]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize

In [8]:
from rank_bm25 import BM25Okapi
from gensim.summarization.bm25 import get_bm25_weights

In [9]:
stop = stopwords.words('russian')

Тексты были взяты с сайта газеты "Известия" (ключевые слова располагаются вверху, над текстом статьи):
https://iz.ru/1241783/evgeniia-priemskaia/dar-rechi-kto-sokhraniaet-iazyki-korennykh-narodov
https://iz.ru/1241275/sergei-sychev/vse-bezhish-forrest
https://iz.ru/1244125/aleksei-zabrodin/liubiat-ne-liubiat
https://iz.ru/1239085/dmitrii-orlov/sokhranit-vs-izmeneniia


Отобранные ключевые слова (я объединила со своими и убрала некоторые, которые были на сайте)
Изначально были такие (на сайте):
1.национальные языки, народы, сибирь, дальний восток
2.кино, кинопремьеры
3.джо байден, сша, рейтинги
4.владимир путин, валдайский клуб, реджеп эрдоган, джо байден


In [43]:
key_words = ['национальные языки, народы, сибирь, дальний восток, коренные малочисленные народы, коренные народы, родной язык, проект "энцы", хабаровский край, язык, народы, эвенкийский язык',
'фильм, Финч, Том Хэнкс, робот',             
'джо байден, сша, рейтинг, политика, белый дом, президент, демократия, демократический',
'владимир путин, страна, политика, государство, клуб Валдай, 2021 год, Россия'             
]

In [11]:
with open('text1.txt', 'r', encoding='utf-8') as f:
  text1 = f.read()
with open('text2.txt', 'r', encoding='utf-8') as f:
  text2 = f.read()
with open('text3.txt', 'r', encoding='utf-8') as f:
  text3 = f.read()
with open('text4.txt', 'r', encoding='utf-8') as f:
  text4 = f.read()

In [12]:
m = MorphAnalyzer()
def normalize_text(text1):
    lemmas = []
    for t in simple_word_tokenize(text1):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [44]:
norm_key_words = []
for i in key_words:
  num = []
  i = normalize_text(i)
  num.append (i)
  norm_key_words.append(num)
print(norm_key_words)

[['национальный язык , народ , сибирь , дальний восток , коренной малочисленный народ , коренной народ , родный язык , проект " энца " , хабаровский край , язык , народ , эвенкийский язык'], ['фильм , финч , тот хэнкс , робот'], ['джо байден , сша , рейтинг , политика , белый дом , президент , демократия , демократический'], ['владимир путин , страна , политика , государство , клуб валдай , 2021 год , россия']]


In [106]:
def raking(text):
  stop.append('который')
  rake = RAKE.Rake(stop)
  text_list = rake.run(normalize_text(text), maxWords=2, minFrequency=2)
  return text_list
rake1 = raking(text1)[:12]
rake2 =raking(text2)[:4]
rake3 =raking(text3)[:8]
rake4 =raking(text4)[:7]
print(rake1)

[('хабаровский край', 4.0), ('учебный пособие', 4.0), ('дальний восток', 4.0), ('рабочий тетрадь', 4.0), ('эвенкийский язык', 3.6875), ('это заниматься', 3.666666666666667), ('изучение язык', 3.520833333333333), ('язык', 1.6875), ('это', 1.6666666666666667), ('учебник', 1.5454545454545454), ('проживать', 1.3333333333333333), ('численность', 1.3333333333333333)]


In [107]:
key1 = keywords.keywords(normalize_text(text1), language='russian', additional_stopwords=stop, scores=True)[:12]
key2 = keywords.keywords(normalize_text(text2), language='russian', additional_stopwords=stop, scores=True)[:4]
key3 = keywords.keywords(normalize_text(text3), language='russian', additional_stopwords=stop, scores=True)[:8]
key4 = keywords.keywords(normalize_text(text4), language='russian', additional_stopwords=stop, scores=True)[:7]

In [16]:
!pip install rutermextract

  Created wheel for rutermextract: filename=rutermextract-0.3-py3-none-any.whl size=9722 sha256=b7691ff4c70cc9c593bbe061154fbbf74fd8db6fb99ae1cba9ebe30f2afb3010
  Stored in directory: /root/.cache/pip/wheels/23/0a/8d/810c38b53b6bfe00a9d964bba36077c633a717e5240883ccbf
Successfully built rutermextract


In [17]:
from rutermextract import TermExtractor
def rutermextract(text):
  term_extractor = TermExtractor()
  key_terms = []
  for term in term_extractor(text):
    termin = []
    termin.append(term.normalized)
    termin.append(term.count)
    key_terms.append(termin)
  return key_terms[:20]

In [108]:
rut1 = rutermextract(normalize_text(text1))[:12]
rut2 = rutermextract(normalize_text(text2))[:4]
rut3 = rutermextract(normalize_text(text3))[:8]
rut4 = rutermextract(normalize_text(text4))[:7]

In [49]:
samples = [['ADJF', 'NOUN'],['NOUN', 'ADJF'],['NOUN', 'NOUN', 'NOUN'],['NOUN', 'NOUN'],['NOUN']]
def right_samples(key_words, samples): 
    filtered = [] 
    for i in key_words: 
        right_samples = [] 
        m = MorphAnalyzer() 
        p = m.parse(i)[0] 
        pos = p.tag.POS 
        right_samples.append(pos) 
        for sample in samples: 
            if right_samples == sample: 
                filtered.append(i) 
    return filtered


In [51]:
list_of_samples = right_samples(norm_key_words[0], samples)
print(list_of_samples)

['национальный язык , народ , сибирь , дальний восток , коренной малочисленный народ , коренной народ , родный язык , проект " энца " , хабаровский край , язык , народ , эвенкийский язык']


In [134]:
def score(my_keywords, result):
    print(my_keywords[0])
    total_precision = 0
    total_recall = 0
    for i, tokens in enumerate(result):
        n = 0
        for token in tokens:
            token = str(token).split(',')
            if str(token[0]) in str(my_keywords[0]):
                n += 1
        recall = n / len(my_keywords)
        precision = n / len(tokens)

        total_precision += precision
        total_recall += recall
    return precision, recall
    
score(norm_key_words[0],key1)

национальный язык , народ , сибирь , дальний восток , коренной малочисленный народ , коренной народ , родный язык , проект " энца " , хабаровский край , язык , народ , эвенкийский язык


(0.0, 0.0)